# Seminar BA & QM
### Code Bol.com Case - Data Preparation

Group 8

****

In [118]:
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import time
import functions
import importlib

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Data Preparation
#### Load data and set data-types 

In [55]:
#Import data
df_2019 = pd.read_csv('/Users/LV/Documents/ECONOMETRIE-ECO/MASTER/Seminar/2021_Case_Bol.com_Happy_Matches/data/data_2019.csv', low_memory = False)
df_2020 = pd.read_csv('/Users/LV/Documents/ECONOMETRIE-ECO/MASTER/Seminar/2021_Case_Bol.com_Happy_Matches/data/data_2020.csv', low_memory = False)

#Concat files and create new index
df_full = pd.concat([df_2019, df_2020])
df_full = df_full.reset_index(drop = True)

print('Total # records: ',df_full.shape[0])

Total # records:  4779466


In [56]:
#Rename columns
rename_columns = {'datetTimeFirstDeliveryMoment': 'dateTimeFirstDeliveryMoment',
                  'generalMatchClassification'  : 'detailedMatchClassification',
                  'detailedMatchClassification' : 'generalMatchClassification',
                  'quanityReturned'             : 'quantityReturned'}

df_full = df_full.rename(columns = rename_columns)

In [57]:
#Transform dates to date-type
df_full['orderDate']                   = pd.to_datetime(df_full['orderDate'])
df_full['cancellationDate']            = pd.to_datetime(df_full['cancellationDate'])
df_full['promisedDeliveryDate']        = pd.to_datetime(df_full['promisedDeliveryDate'])
df_full['shipmentDate']                = pd.to_datetime(df_full['shipmentDate'])
df_full['dateTimeFirstDeliveryMoment'] = pd.to_datetime(df_full['dateTimeFirstDeliveryMoment'])
df_full['startDateCase']               = pd.to_datetime(df_full['startDateCase'])
df_full['returnDateTime']              = pd.to_datetime(df_full['returnDateTime'])
df_full['registrationDateSeller']      = pd.to_datetime(df_full['registrationDateSeller'])

#Change type of columns
dtype = {'calculationDefinitive': bool,
         'noCancellation'       : bool,
         'noCase'               : bool,
         'hasOneCase'           : bool,
         'hasMoreCases'         : bool,
         'noReturn'             : bool}

df_full = df_full.astype(dtype)

#### Remove Noise

In [58]:
#Remove non-sensible rows
noise = df_full.loc[(df_full['startDateCase']        < df_full['orderDate']) | 
                   (df_full['cancellationDate']      < df_full['orderDate']) |
                   (df_full['promisedDeliveryDate']  < df_full['orderDate']) |
                   (df_full['shipmentDate']          < df_full['orderDate']) |
                   (df_full['returnDateTime']        < df_full['orderDate']) |
                   (df_full['cancellationDate']      > df_full['returnDateTime']) |
                   (df_full['shipmentDate']          > df_full['returnDateTime']) |
                   (df_full['orderDate']             < df_full['registrationDateSeller']) |
                   (df_full['orderDate']             > df_full['dateTimeFirstDeliveryMoment'].dt.normalize()) |
                   (df_full['cancellationDate']      > df_full['dateTimeFirstDeliveryMoment'].dt.normalize()) |
                   (df_full['shipmentDate']          > df_full['dateTimeFirstDeliveryMoment'].dt.normalize()) |
                   ((df_full['returnDateTime']       < df_full['dateTimeFirstDeliveryMoment'].dt.normalize()) & 
                                                       (df_full['dateTimeFirstDeliveryMoment'].notnull()) &
                                                       (df_full['returnDateTime'].notnull())) |
                   ((df_full['cancellationDate']     > df_full['shipmentDate']) &
                                                       ((df_full['cancellationReasonCode'] == 'CUST_FE') |
                                                       (df_full['cancellationReasonCode'] == 'CUST_CS'))) |
                   (df_full['registrationDateSeller'].isnull()) |
                   (df_full['promisedDeliveryDate'].isnull())].index

#Drop noise data
df = df_full.drop(index = noise)
print(len(noise), 'complete records removed from the data')
print('Cleaned # records: ',df.shape[0],'\n')

#Sort rows on orderDate and create new index
df = df.sort_values(by = 'orderDate')
df = df.reset_index(drop = True)

#Remove inconsistent values
cancellationNoise = df.loc[(df['noCancellation'] == True) & (df['cancellationDate'].notnull())].index
returnNoise       = df.loc[(df['noReturn'] == True) & (df['returnDateTime'].notnull())].index
caseNoise         = df.loc[(df['noCase'] == True) & (df['startDateCase'].notnull())].index
quantityNoise     = df.loc[df['quantityReturned'] > df['quantityOrdered']].index
deliveryNoise     = df.loc[(df['dateTimeFirstDeliveryMoment'].notnull()) & (df['onTimeDelivery'].isnull())].index

df.loc[cancellationNoise, ['cancellationDate','cancellationReasonCode']] = None
df.loc[returnNoise,       ['returnDateTime','quantityReturned','returnCode']] = None
df.loc[caseNoise,         ['startDateCase','cntDistinctCaseIds','hasOneCase','hasMoreCases']] = None
df.loc[quantityNoise,     ['quantityReturned']] = df.loc[quantityNoise, ['quantityOrdered']]
df.loc[deliveryNoise,     ['dateTimeFirstDeliveryMoment']] = None

print('# Records where cancellation values are emptied:',len(cancellationNoise))
print('# Records where return values are emptied: \t',len(returnNoise))
print('# Records where case values are emptied: \t',len(caseNoise))
print('# Records where quantity values are equalized: \t',len(quantityNoise))
print('# Records where delivery values are emptied: \t',len(deliveryNoise))

6516 complete records removed from the data
Cleaned # records:  4772950 

# Records where cancellation values are emptied: 53780
# Records where return values are emptied: 	 8208
# Records where case values are emptied: 	 0
# Records where quantity values are equalized: 	 14722
# Records where delivery values are emptied: 	 4103


#### Create New Variables

##### Time Related

In [59]:
#Create new variables related to periods (days difference)
df['caseDays']             = (df['startDateCase'] - df['orderDate']).dt.days
df['returnDays']           = (df['returnDateTime'] - df['orderDate']).dt.days
df['cancellationDays']     = (df['cancellationDate'] - df['orderDate']).dt.days
df['actualDeliveryDays']   = (df['dateTimeFirstDeliveryMoment'].dt.normalize() - df['orderDate']).dt.days
df['shipmentDays']         = (df['shipmentDate'] - df['orderDate']).dt.days
df['partnerSellingMonths'] = (df['orderDate'] - df['registrationDateSeller']).dt.days
df['promisedDeliveryDays'] = (df['promisedDeliveryDate'] - df['orderDate']).dt.days

#Time related variables
df['orderYear']    = df['orderDate'].dt.year
df['orderMonth']   = df['orderDate'].dt.month
df['orderWeekday'] = df['orderDate'].dt.weekday
df['orderCorona']  = df['orderDate'].apply(lambda x: True if x > datetime.strptime('2020-03-20','%Y-%m-%d') else False)

#Create dummy variables for weekdays, months and years
df['orderMonday']    = df['orderWeekday'].apply(lambda x: True if x == 1 else False)
df['orderTuesday']   = df['orderWeekday'].apply(lambda x: True if x == 2 else False)
df['orderWednesday'] = df['orderWeekday'].apply(lambda x: True if x == 3 else False)
df['orderThursday']  = df['orderWeekday'].apply(lambda x: True if x == 4 else False)
df['orderFriday']    = df['orderWeekday'].apply(lambda x: True if x == 5 else False)
df['orderSaturday']  = df['orderWeekday'].apply(lambda x: True if x == 6 else False)
df['orderSunday']    = df['orderWeekday'].apply(lambda x: True if x == 7 else False)

df['orderJanuary']   = df['orderMonth'].apply(lambda x: True if x == 1 else False)
df['orderFebruary']  = df['orderMonth'].apply(lambda x: True if x == 2 else False)
df['orderMarch']     = df['orderMonth'].apply(lambda x: True if x == 3 else False)
df['orderApril']     = df['orderMonth'].apply(lambda x: True if x == 4 else False)
df['orderMay']       = df['orderMonth'].apply(lambda x: True if x == 5 else False)
df['orderJune']      = df['orderMonth'].apply(lambda x: True if x == 6 else False)
df['orderJuly']      = df['orderMonth'].apply(lambda x: True if x == 7 else False)
df['orderAugust']    = df['orderMonth'].apply(lambda x: True if x == 8 else False)
df['orderSeptember'] = df['orderMonth'].apply(lambda x: True if x == 9 else False)
df['orderOctober']   = df['orderMonth'].apply(lambda x: True if x == 10 else False)
df['orderNovember']  = df['orderMonth'].apply(lambda x: True if x == 11 else False)
df['orderDecember']  = df['orderMonth'].apply(lambda x: True if x == 12 else False)

df['orderYear2019'] = df['orderYear'].apply(lambda x: True if x == 2019 else False)
df['orderYear2020'] = df['orderYear'].apply(lambda x: True if x == 2020 else False)

##### Other

In [60]:
df['productTitleLength'] = len(df['productTitle'])

df['fulfilmentByBol'] = df['fulfilmentType'].apply(lambda x: True if x == 'FBB' else False)

df['countryCodeNL']   = df['countryCode'].apply(lambda x: True if x == 'NL' else False)
df['countryOriginNL'] = df['countryOriginSeller'].apply(lambda x: True if x == 'NL' else False)
df['countryOriginBE'] = df['countryOriginSeller'].apply(lambda x: True if x == 'BE' else False)
df['countryOriginDE'] = df['countryOriginSeller'].apply(lambda x: True if x == 'DE' else False)

##### Determinant Classification

In [61]:
def determinantClassification(df):
    """
    Function to create column with determinants for each order.
    """
    if (df.noCancellation == 1 and df.noReturn == 1 and df.noCase == 1 and df.onTimeDelivery == True): 
        return 'All good'
    elif (df.noCancellation == 1 and df.noReturn == 1 and df.noCase == 1 and df.onTimeDelivery == None):
        return 'Unknown delivery'
    elif (df.noCancellation == 1 and df.noReturn == 1 and df.noCase == 1 and df.onTimeDelivery == False):
        return 'Late delivery'
    elif (df.noCancellation == 1 and df.noReturn == 1 and df.noCase == 0 and df.onTimeDelivery == True):
        return 'Case'
    elif (df.noCancellation == 1 and df.noReturn == 1 and df.noCase == 0 and df.onTimeDelivery == None):
        return 'Case + Unknown delivery'
    elif (df.noCancellation == 1 and df.noReturn == 1 and df.noCase == 0 and df.onTimeDelivery == False):
        return 'Case + Late delivery'
    elif (df.noCancellation == 1 and df.noReturn == 0 and df.noCase == 1 and df.onTimeDelivery == True):
        return 'Return'
    elif (df.noCancellation == 1 and df.noReturn == 0 and df.noCase == 1 and df.onTimeDelivery == None):
        return 'Return + Unknown delivery'
    elif (df.noCancellation == 1 and df.noReturn == 0 and df.noCase == 1 and df.onTimeDelivery == False):
        return 'Return + Late delivery'
    elif (df.noCancellation == 1 and df.noReturn == 0 and df.noCase == 0 and df.onTimeDelivery == True):
        return 'Return + Case'
    elif (df.noCancellation == 1 and df.noReturn == 0 and df.noCase == 0 and df.onTimeDelivery == None):
        return 'Return + Case + Unknown delivery'
    elif (df.noCancellation == 1 and df.noReturn == 0 and df.noCase == 0 and df.onTimeDelivery == False):
        return 'Return + Case + Late delivery'
    elif (df.noCancellation == 0 and df.noReturn == 1 and df.noCase == 0 and df.onTimeDelivery == True):
        return 'Cancellation + Case'
    elif (df.noCancellation == 0):
        return 'Cancellation'

In [62]:
df['determinantClassification'] = df.apply(determinantClassification, axis = 1)
df['determinantClassification'].value_counts()

All good                         2685606
Late delivery                     146725
Return                            145556
Case                               42643
Return + Case                      24204
Cancellation                       23690
Return + Late delivery              8426
Case + Late delivery                8352
Return + Case + Late delivery       2134
Cancellation + Case                    3
Name: determinantClassification, dtype: int64

##### Binary Match Classification

In [63]:
df['binaryMatchClassification'] = df['generalMatchClassification'].apply(lambda x: 'UNKNOWN' if x == 'UNKNOWN' else 'KNOWN')

##### Transporter

In [64]:
def transporterCluster(transporterCode):
    """
    Function to create a new manually clustered transporter variable: 28 -> 5 categories
    """
    if transporterCode in ['AH-NL','TNT','TNT-EXPRESS','TNT-EXTRA']:
        return 'POSTNL'
    elif transporterCode in ['DHL','DHL_DE','DHLFORYOU']:
        return 'DHL'
    elif transporterCode in ['DPD-NL','DPD-BE']:
        return 'DPD'
    elif transporterCode in ['BRIEFPOST','BPOST_BRIEF','DHL-GLOBAL-MAIL','TNT_BRIEF']:
        return 'BRIEF'
    else:
        return 'OTHER'

In [65]:
df['transporterCodeGeneral'] = df['transporterCode'].apply(transporterCluster)
df['transporterCodeGeneral'].value_counts()

POSTNL    2110753
BRIEF     1576462
DHL        436975
DPD        329746
OTHER      319014
Name: transporterCodeGeneral, dtype: int64

##### Product Group

In [66]:
def productGroupCluster(productGroup):
    """
    Function to create a new manually clustered product group variable based on categories bol.com
    60 -> 14 groups.
    """
    if productGroup in ['Dutch Books PG','Ebooks and Audiobooks','International Books PG']:
        return 'Books'
    elif productGroup in ['Games Accessories','Games Consoles','Games Software Physical',
                          'Movies','Music']:
        return 'Music, Film & Games'
    elif productGroup in ['Camera','Desktop Monitor and Beamer','Ereaders and Accessories',
                          'Laptop Computers','PC Accessories','Personal Audio',
                          'Sound and Vision Accessories','Storage and Network',
                          'Telephone and Tablet Accessories','Telephones and Tablets','Television']:
        return 'Computer & Electronics'
    elif productGroup in ['General Toys','Recreational and Outdoor Toys']:
        return 'Toys & Hobby'
    elif productGroup in ['Baby and Kids Fashion','Baby PG']:
        return 'Baby & Kids'
    elif productGroup in ['Daily Care PG','Health PG','Perfumery PG','Personal Care']:
        return 'Health & Care'
    elif productGroup in ['Footwear','Jewelry and Watches','Mens and Womens Fashion','Wearables']:
        return 'Fashion, Shoes & Accessories'
    elif productGroup in ['Bodyfashion and Beachwear','Camping and Outdoor','Cycling',
                          'Sporting Equipment','Sportswear','Travel Bags and Accessories']:
        return 'Sports, Outdoor & Travel'
    elif productGroup in ['Educational Dutch','Educational International','Printing and Ink']:
        return 'Office & School'
    elif productGroup in ['Supermarket PG'] :
        return 'Food & Beverage'
    elif productGroup in ['Furniture','Heating and Air','Home Decoration','Home Entertainment',
                          'Household','Household Appliances','Kitchen','Kitchen Machines',
                          'Lighting','Major Domestic Appliances PG','Plumbing and Safety']:
        return 'Home, Cooking & Household'
    elif productGroup in ['Garden','Pet PG','Textiles','Tools and Paint']:
        return 'Pets, Garden & Jobs'
    elif productGroup in ['Car and Motorcycle'] :
        return 'Car & Motor'
    else:
        return 'Other'

In [67]:
df['productGroupGeneral'] = df['productGroup'].apply(productGroupCluster)
df['productGroupGeneral'].value_counts()

Computer & Electronics          1387679
Home, Cooking & Household        797874
Sports, Outdoor & Travel         522098
Toys & Hobby                     500977
Pets, Garden & Jobs              339813
Health & Care                    299049
Food & Beverage                  258769
Books                            184581
Music, Film & Games              163842
Baby & Kids                      113707
Fashion, Shoes & Accessories     110067
Office & School                   52270
Car & Motor                       29753
Other                             12471
Name: productGroupGeneral, dtype: int64

In [68]:
#Create dummies of new product grouping
for group in df['productGroupGeneral'].unique():
    
    columnName = 'group' + group.split(' ')[0].replace(',','')
    df[columnName] = df['productGroupGeneral'].apply(lambda x: True if x == group else False)

##### Total Columns

In [69]:
print(df.columns)
print('Total: ',len(df.columns),' columns')

Index(['orderDate', 'productId', 'sellerId', 'totalPrice', 'quantityOrdered',
       'countryCode', 'cancellationDate', 'cancellationReasonCode',
       'promisedDeliveryDate', 'shipmentDate', 'transporterCode',
       'transporterName', 'transporterNameOther',
       'dateTimeFirstDeliveryMoment', 'fulfilmentType', 'startDateCase',
       'cntDistinctCaseIds', 'returnDateTime', 'quantityReturned',
       'returnCode', 'productTitle', 'brickName', 'chunkName', 'productGroup',
       'productSubGroup', 'productSubSubGroup', 'registrationDateSeller',
       'countryOriginSeller', 'currentCountryAvailabilitySeller',
       'calculationDefinitive', 'noCancellation', 'onTimeDelivery', 'noCase',
       'hasOneCase', 'hasMoreCases', 'noReturn', 'detailedMatchClassification',
       'generalMatchClassification', 'caseDays', 'returnDays',
       'cancellationDays', 'actualDeliveryDays', 'shipmentDays',
       'partnerSellingMonths', 'promisedDeliveryDays', 'orderYear',
       'orderMonth', 'ord

In [78]:
#Fixed Columns:
DATE = ['orderDate']
BASIC = ['totalPrice','quantityOrdered','fulfilmentByBol','countryCodeNL','countryOriginNL','countryOriginBE',
        'countryOriginDE','productTitleLength','promisedDeliveryDays','partnerSellingMonths']
WEEK = ['orderMonday','orderTuesday','orderWednesday','orderThursday','orderFriday','orderSaturday','orderSunday']
MONTH = ['orderJanuary','orderFebruary','orderMarch','orderApril','orderMay','orderJune',
         'orderJuly','orderAugust','orderSeptember','orderOctober','orderNovember','orderDecember']
YEAR = ['orderYear2019','orderYear2020']
GROUP = ['groupHealth','groupHome','groupSports','groupComputer','groupPets','groupToys','groupBooks', 
         'groupBaby', 'groupMusic', 'groupFood','groupOffice','groupFashion','groupOther','groupCar']

#Dynamic Columns:
TRANSPORTERX = ['transporterPOSTNL/X','transporterDHL/X','transporterDPD/X','transporterBRIEF/X','transporterOTHER/X']
KNOWNX = ['caseKnownX','returnKnownX','cancellationKnownX','onTimeDeliveryKnownX','lateDeliveryKnownX']
PRODUCTX = ['productOrderCountX','productTotalCountX','productTotalReturnedX','productReturnFractionX']
SELLERX = ['sellerDailyOrdersX']

#Classifications
CLASSIFICATION = ['generalMatchClassification','detailedMatchClassification','binaryMatchClassification','determinantClassification']

In [70]:
# Convert to CSV File
# df.to_csv('/Users/LV/Desktop/data_bol_complete.csv',index = False)

#### TESTING

In [37]:
#df.loc[(df['productId'] == 9300000000762808) | (df['productId'] == 9200000101821794)]
#df.loc[(df['productId'] == 9200000105019922)]
#[['productId','orderDate','returnDateTime','quantityOrdered','quantityReturned']]
#[['caseDays','returnDays','cancellationDays','actualDeliveryDays']]

# t0 = time.time()
# a = functions.addProductColumns(df)
# b = functions.addProductColumnsX(a,2)
# t1 = time.time()
# total = t1-t0
# print(total)

In [128]:
# df_ = df.loc[(df['productId'] == 9300000000762808) | (df['productId'] == 9200000101821794)]
df_ = df.loc[(df['productId'] == 9200000105019922)]
# df_ = df.loc[df['sellerId'] == 1003411]
df_ = df_[['productId','orderDate','returnDateTime','quantityOrdered','quantityReturned',
           'caseDays','returnDays','cancellationDays','actualDeliveryDays','onTimeDelivery',
           'transporterCodeGeneral','shipmentDays']]
# df_ = df_[['productId','orderDate','productTotalCount','productOrderCount','productTotalReturned']]
# df_ = df_[['sellerId','orderDate','quantityOrdered','partnerSellingMonths']]

#df_ = functions.addProductColumns(df_,0)
df_ = functions.addKnownColumns(df_,0)
df_

,productId,orderDate,returnDateTime,quantityOrdered,quantityReturned,caseDays,returnDays,cancellationDays,actualDeliveryDays,onTimeDelivery,transporterCodeGeneral,shipmentDays,caseKnownX,returnKnownX,cancellationKnownX,onTimeDeliveryKnownX,lateDeliveryKnownX,transporterPOSTNL/X,transporterBRIEF/X,transporterOTHER/X
834922,9200000105019922,2019-07-05,2019-07-09,1,1.0,NaN,4.0,NaN,3.0,True,POSTNL,1.0,False,False,False,False,False,False,False,False
959571,9200000105019922,2019-07-30,NaT,1,NaN,NaN,NaN,NaN,1.0,True,POSTNL,0.0,False,False,False,False,False,True,False,False
961389,9200000105019922,2019-07-30,NaT,1,NaN,NaN,NaN,NaN,NaN,NaN,BRIEF,1.0,False,False,False,False,False,False,False,False
963833,9200000105019922,2019-07-30,NaT,1,NaN,NaN,NaN,NaN,NaN,NaN,BRIEF,1.0,False,False,False,False,False,False,False,False
964578,9200000105019922,2019-07-30,NaT,1,NaN,NaN,NaN,NaN,1.0,True,POSTNL,0.0,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4632775,9200000105019922,2020-12-08,NaT,1,NaN,NaN,NaN,NaN,1.0,True,POSTNL,0.0,False,False,False,False,False,True,False,False
4648104,9200000105019922,2020-12-09,NaT,1,NaN,NaN,NaN,NaN,NaN,NaN,BRIEF,0.0,False,False,False,False,False,False,True,False
4673640,9200000105019922,2020-12-11,NaT,2,NaN,NaN,NaN,NaN,1.0,True,POSTNL,0.0,False,False,False,False,False,True,False,False
4731260,9200000105019922,2020-12-15,NaT,1,NaN,NaN,NaN,NaN,NaN,NaN,BRIEF,0.0,False,False,False,False,False,False,True,False
